<a href="https://colab.research.google.com/github/alfonsomayoral/8_Puzzle_Code/blob/main/CS170_Project1_Alfonso_Mayoral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Eight Puzzle Problem**

In [3]:
import heapq
import copy
import time

# Define the goal state of the puzzle as a 3x3 grid where the tiles are in order
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

def find_zero(state):
    """Find the position of the empty tile (0) in the puzzle."""
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == 0: # Identify the position of the blank tile
                return i, j

def is_goal(state):
    """Check if the current state matches the goal state."""
    return state == goal_state

def get_neighbors(state):
    """Generate all possible moves by sliding tiles adjacent to the empty space."""
    x, y = find_zero(state)
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = copy.deepcopy(state)
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)

    return neighbors

def misplaced_tile_heuristic(state):
    """Calculate the number of tiles that are not in their correct positions."""
    count = 0
    for i in range(3):
        for j in range(3):
            # Exclude the blank tile and count tiles that are not in their goal positions
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                count += 1
    return count

def manhattan_distance_heuristic(state):
    """Calculate the sum of Manhattan distances of tiles from their goal positions."""
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                goal_x, goal_y = divmod(state[i][j] - 1, 3)
                distance += abs(goal_x - i) + abs(goal_y - j)
    return distance

def general_search(initial_state, heuristic_fn):
    """General search algorithm that expands nodes based on a given heuristic."""
    start_time = time.time()
    pq = []
    heapq.heappush(pq, (0, 0, initial_state, []))  # (f(n), g(n), state, path)
    visited = set()
    max_queue_size = 0
    nodes_expanded = 0

    while pq:
        max_queue_size = max(max_queue_size, len(pq))
        fn, gn, current_state, path = heapq.heappop(pq)

        if tuple(map(tuple, current_state)) in visited:
            continue

        visited.add(tuple(map(tuple, current_state)))

        if is_goal(current_state):
            elapsed_time = time.time() - start_time # End timing
            return path, gn, nodes_expanded, max_queue_size, elapsed_time, len(visited)

        nodes_expanded += 1

        for neighbor in get_neighbors(current_state):
            new_path = path + [(neighbor, gn + 1, heuristic_fn(neighbor))]  # Store g(n) and h(n)
            hn = heuristic_fn(neighbor)
            heapq.heappush(pq, (gn + 1 + hn, gn + 1, neighbor, new_path))

    elapsed_time = time.time() - start_time
    return "Failure", -1, nodes_expanded, max_queue_size, elapsed_time, len(visited)

def uniform_cost_search(initial_state):
    """Algorithm 1: Perform Uniform Cost Search."""
    return general_search(initial_state, lambda _: 0) # No heuristic (h(n) = 0)

def a_star_misplaced_tile(initial_state):
    """Algorithm 2: Perform A* search using the Misplaced Tile heuristic."""
    return general_search(initial_state, misplaced_tile_heuristic)

def a_star_manhattan_distance(initial_state):
    """Algorithm 3: Perform A* search using the Manhattan Distance heuristic."""
    return general_search(initial_state, manhattan_distance_heuristic)

if __name__ == "__main__":
    print("Welcome to the 8-Puzzle Solver!")
    choice = input("Type '1' to use a default puzzle, or '2' to input your own: ")

    if choice == '1':
        initial_state = [[8, 6, 7], [2, 5, 4], [3, 0, 1]]
    else:
        print("Enter your puzzle (3x3 grid, use space to separate numbers, and 0 for blank):")
        initial_state = [list(map(int, input(f"Enter row {i + 1}: ").split())) for i in range(3)]

    print("Select the algorithm:")
    print("1. Uniform Cost Search")
    print("2. A* with Misplaced Tile Heuristic")
    print("3. A* with Manhattan Distance Heuristic")

    # Prompt the user to select a search algorithm
    algorithm_choice = input("Enter your choice (1/2/3): ") # Example default state, depth 31

    if algorithm_choice == '1':
        path, cost, nodes_expanded, max_queue_size, elapsed_time, visited_states = uniform_cost_search(initial_state)
    elif algorithm_choice == '2':
        path, cost, nodes_expanded, max_queue_size, elapsed_time, visited_states = a_star_misplaced_tile(initial_state)
    elif algorithm_choice == '3':
        path, cost, nodes_expanded, max_queue_size, elapsed_time, visited_states = a_star_manhattan_distance(initial_state)
    else:
        print("Invalid choice. Exiting.")
        exit()

    if path != "Failure":
        print("Solution found!")
        print("Steps to solution:")
        for step, g_n, h_n in path:
            for row in step:
                print(row)  #Print each state in the solution path
            print(f"g(n): {g_n}, h(n): {h_n}\n")

        print(f"Depth: {cost}") # Print the depth of the solution
        print(f"Nodes expanded: {nodes_expanded}")  # Print the total nodes expanded
        print(f"Time taken: {elapsed_time:.4f} seconds")  # Print the time required to reach the goal state
        print(f"Maximum queue size: {max_queue_size}")  # Print the max queue size
        print(f"Total visited states: {visited_states}") # Print the total nodes visited
    else:
        print("No solution found.")

Welcome to the 8-Puzzle Solver!
Type '1' to use a default puzzle, or '2' to input your own: 1
Select the algorithm:
1. Uniform Cost Search
2. A* with Misplaced Tile Heuristic
3. A* with Manhattan Distance Heuristic
Enter your choice (1/2/3): 2
Solution found!
Steps to solution:
[8, 6, 7]
[2, 0, 4]
[3, 5, 1]
g(n): 1, h(n): 8

[8, 0, 7]
[2, 6, 4]
[3, 5, 1]
g(n): 2, h(n): 8

[0, 8, 7]
[2, 6, 4]
[3, 5, 1]
g(n): 3, h(n): 8

[2, 8, 7]
[0, 6, 4]
[3, 5, 1]
g(n): 4, h(n): 8

[2, 8, 7]
[3, 6, 4]
[0, 5, 1]
g(n): 5, h(n): 8

[2, 8, 7]
[3, 6, 4]
[5, 0, 1]
g(n): 6, h(n): 8

[2, 8, 7]
[3, 6, 4]
[5, 1, 0]
g(n): 7, h(n): 8

[2, 8, 7]
[3, 6, 0]
[5, 1, 4]
g(n): 8, h(n): 8

[2, 8, 0]
[3, 6, 7]
[5, 1, 4]
g(n): 9, h(n): 8

[2, 0, 8]
[3, 6, 7]
[5, 1, 4]
g(n): 10, h(n): 8

[2, 6, 8]
[3, 0, 7]
[5, 1, 4]
g(n): 11, h(n): 8

[2, 6, 8]
[0, 3, 7]
[5, 1, 4]
g(n): 12, h(n): 8

[2, 6, 8]
[5, 3, 7]
[0, 1, 4]
g(n): 13, h(n): 8

[2, 6, 8]
[5, 3, 7]
[1, 0, 4]
g(n): 14, h(n): 8

[2, 6, 8]
[5, 3, 7]
[1, 4, 0]
g(n): 15, h(n)